# 推薦システムにおける行列分解の話

## 概要

この記事では、推薦システムにおける行列分解の手法について解説する。行列分解の定義や性質、応用例を具体的な数式とPythonのコードを用いて示す。また、行列分解のメリットとデメリットについても論じ、具体的な利用例として「movielens-100k」データセットを用いた実装例を紹介する。



## ソースコード


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/gr/05/05_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/gr/05/05_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なりますので注意していただきたい。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行います。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



## 行列分解の定義と性質

行列分解(Matrix Factorization)は、与えられた行列を二つの低ランク行列に分解する手法である。推薦システムでは、ユーザーとアイテムの行列を分解することで、潜在因子を抽出し、推薦を行う。

### 数式表現

ユーザー $u$ とアイテム $i$ の評価行列 $\mathbf{R}$ を次のように分解する。

$$
\mathbf{R} \approx \mathbf{P} \mathbf{Q}^T
$$

ここで、$\mathbf{P}$ はユーザー行列、$\mathbf{Q}$ はアイテム行列である。各行列の次元は以下の通りである。

$$
\mathbf{P} \in \mathbb{R}^{m \times k}, \quad \mathbf{Q} \in \mathbb{R}^{n \times k}
$$

ここで、$m$ はユーザー数、$n$ はアイテム数、$k$ は潜在因子の次元である。目標は、評価行列 $\mathbf{R}$ と予測行列 $\mathbf{P} \mathbf{Q}^T$ との差の二乗和を最小化することである。

### 最適化問題

最適化問題は以下のように定式化される。

$$
\min_{\mathbf{P}, \mathbf{Q}} \sum_{(u,i) \in \mathcal{K}} \left( r_{ui} - \mathbf{p}_u \cdot \mathbf{q}_i^T \right)^2 + \lambda \left( \|\mathbf{p}_u\|^2 + \|\mathbf{q}_i\|^2 \right)
$$

ここで、$\mathcal{K}$ は評価が存在するユーザーとアイテムのペアの集合、$\lambda$ は正則化パラメータである。この正則化項により、過学習を防ぐ。

## 応用例と実装

行列分解は、様々な推薦システムに応用されている。以下に、具体的な応用例として「movielens-100k」データセットを用いた実装を示す。

### データセットの準備

まず、「movielens-100k」データセットをロードし、評価行列を準備する。




In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from pprint import pprint

# データセットの読み込み
ratings = pd.read_csv("u.data", sep="\t", header=None, names=["user_id", "movie_id", "rating", "timestamp"])
ratings = ratings.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)

# 評価行列の作成
R = ratings.values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

### 行列分解の実装

次に、評価行列を行列分解する。ここでは、SVD（特異値分解）を用いる。

In [ ]:
# 特異値分解
U, sigma, Vt = svds(R_demeaned, k=50)
sigma = np.diag(sigma)

# 予測行列の作成
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
predicted_ratings = pd.DataFrame(all_user_predicted_ratings, columns=ratings.columns)

# ユーザー1に対する映画推薦
user_id = 1
user_row_number = user_id - 1  # 行番号は0から始まるため
sorted_user_predictions = predicted_ratings.iloc[user_row_number].sort_values(ascending=False)

# 元の評価と予測評価の表示
user_data = ratings.loc[user_id]
user_full = pd.concat([user_data, sorted_user_predictions], axis=1)
user_full.columns = ["Original Rating", "Predicted Rating"]
pprint(user_full.head(10))



このコードでは、評価行列を特異値分解し、予測行列を作成してユーザー1に対する映画の推薦を行う。

## メリットとデメリット

### メリット

- **高精度**: 行列分解は多くの推薦システムで高精度な予測を提供する。
- **スケーラビリティ**: 大規模なデータセットでも効率的に処理可能である。
- **潜在因子の解釈**: 潜在因子により、ユーザーやアイテムの特徴を理解しやすくなる。

### デメリット

- **データの疎性**: 評価行列が疎である場合、分解が難しい。
- **計算コスト**: 行列分解には高い計算コストが伴う。
- **パラメータチューニング**: 正則化パラメータや潜在因子の次元数などのパラメータのチューニングが必要である。

## 結論

この記事では、推薦システムにおける行列分解について詳述した。具体的な定義や数式、Pythonコードを用いた具体例を示し、メリットとデメリットを論じた。この手法は、多くの推薦システムで応用され、高精度な推薦を実現する。


### 参考文献

- Wikipedia: [Matrix Factorization](https://en.wikipedia.org/wiki/Matrix_factorization)
- Scipy: [Singular Value Decomposition](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)